In [1]:
import numpy as np
import pandas as ps
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.models import Model
from keras import backend as K
from keras import metrics

from sklearn.model_selection import train_test_split

In [8]:
df = pd.read_csv("./data/cleaned_data.csv")
df.head()

,index,cell_id,DL_TRAFFIC_VOLUME,UL_TRAFFIC_VOLUME,Inter_X2_based_HO_prep,PDCP_SDU_Volume_DL,VoLTE_total_traffic,INTRA_FREQ_HO_SR_RATIO,RRC_SR_RATIO,Intra_eNB_HO_SR_total_RATIO,...,DCR_LTE_RATIO,CSSR_LTE_RATIO,LTE_INTER_ENODEB_HOSR_RATIO,E_UTRAN_Inter_Freq_HO_SR_RATIO,Inter_RAT_HO_SR_GERAN_SRVCC_RATIO,Inter_RAT_Total_HO_SR_RATIO,E_UTRAN_tot_HO_SR_inter_eNB_X2_RATIO,DL_THROUGHPUT_RATIO,E_RAB_DR_RATIO,HOUR
0,2021-05-09 00:00:00,2.226537e+17,3.779737e+10,3.947172e+09,15.0,3.779737e+10,4727.0,0.809859,0.992427,0.809859,...,0.001761,0.996041,0.400000,0.770642,0.963636,0.963636,0.400000,82.550417,0.001761,0
1,2021-05-09 01:00:00,2.226537e+17,3.684898e+10,4.088752e+09,6.0,3.684898e+10,3076.0,0.886792,0.993288,0.886792,...,0.002468,0.995465,0.500000,0.842105,1.000000,1.000000,0.500000,80.531811,0.002468,1
2,2021-05-09 02:00:00,2.226537e+17,3.292677e+10,5.016897e+09,8.0,3.292677e+10,3501.0,0.938356,0.994664,0.938356,...,0.003077,0.996044,0.375000,0.931624,1.000000,0.966667,0.375000,71.885311,0.003077,2
3,2021-05-09 03:00:00,2.226537e+17,3.021547e+10,5.139107e+09,9.0,3.021547e+10,2275.0,0.860215,0.994819,0.860215,...,0.001721,0.995920,0.777778,0.816901,0.947368,0.947368,0.777778,65.977430,0.001721,3
4,2021-05-09 04:00:00,2.226537e+17,3.082176e+10,4.250716e+09,17.0,3.082176e+10,2178.0,0.840426,0.995952,0.840426,...,0.002213,0.995628,0.764706,0.794521,1.000000,1.000000,0.764706,67.324805,0.002213,4


In [9]:
cell_ids = df["cell_id"].unique()

# A list of dataframe of different cell_id
list_cells = []
for i in range(len(cell_ids)):
    cell_id = cell_ids[i]
    list_cells.append(df[df["cell_id"]==cell_id])

In [11]:
data  = list_cells[1]
shape = data.shape 
shape

(1656, 27)

In [10]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""
    
    def call(self,inputs):
        z_mean, z_log_var = inputs
        batch   = tf.shape(z_mean)[0]
        dim     = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch,dim))
        
        return z_mean + tf.exp(0.5*z_log_var) * epsilon   

In [ ]:
encoder_inputs = keras.Input(shape=shape)
x = layers.Conv1D(8,3,activation="relu")(encoder_inputs)
x = layers.Conv1D(16,3,activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dense()

In [ ]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, args):
        z_mean, z_log_var = args
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
original_dim = x_train.shape[1] # x_train.shape = (1241,24)
latent_dim = 2                  # = dim in Sampling

encoder_input = layers.Input(shape=(1,original_dim))
x = layers.Conv1D(
            filters=32, kernel_size=3, padding="same", strides=2, activation="relu"
        )(encoder_input)
x = layers.Dropout(rate=0.2)(x)
x = layers.Conv1D(
            filters=16, kernel_size=3, padding="same", strides=2, activation="relu"
        )(x)
x = layers.Flatten()(x)
x = layers.Dense(8,activation="relu")(x)

z_mean    = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z         = Sampling()([z_mean,z_log_var])
encoder_output = [z_mean,z_log_var,z]

encoder = keras.Model(encoder_input, encoder_output, name="encoder")
encoder.summary()

In [ ]:
# intermediate_dim = int(original_dim/4)
laten_input = layers.Input(shape=(latent_dim,)) #这里尺寸要跟上面mean的一样

x = layers.Dense(1*16,activation="relu")(laten_input)
x = layers.Reshape((1,16))(x)
x = layers.Conv1DTranspose(
            filters=16, kernel_size=3, padding="same", activation="relu"
        )(x)
x = layers.Dropout(rate=0.2)(x)
x = layers.Conv1DTranspose(
            filters=32, kernel_size=3, padding="same", activation="relu"
        )(x)

decoder_output = layers.Conv1DTranspose(filters=original_dim, kernel_size=3, padding="same",  activation="softmax")(x)
decoder = keras.Model(laten_input,decoder_output, name="decoder")
decoder.summary()

In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.mean_squared_error(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [ ]:
vae = VAE(encoder,decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(df_cell, epochs=30, batch_size=32)